In [1]:
import csv
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split  
import string
import re

# Pemanggilan Dataset

In [2]:
docs = pd.read_csv('book3.csv', header=None, names=['Class', 'Tweet'])
docs.head()

,Class,Tweet
0,anger,SUSAH NGOMONG SAMA ORANG YANG GA TAU DIRI
1,sadness,"macam mw balas tweet klau sy x pandai english,..."
2,sadness,knp ka?? aku jg gk denger siaran trakhir kk lg...
3,fear,hujan lbat + petirr aku #takut pengen off aja
4,fear,"Yang tadinya tenang dirumah,, skrang malah jad..."


In [3]:
kelas=docs.Class.value_counts()
kelas

joy        250
sadness    250
fear       250
anger      250
Name: Class, dtype: int64

In [4]:
X = docs.Tweet
y = docs.Class
print(X.shape)
print(y.shape)

(1000,)
(1000,)


# Preprocessing

In [5]:
def cleaning(data):
    text=[]
    
    tweet= data.lower()
    
         # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
        # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
        # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
        # remove hashtags
        # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
        #remove @
    tweet = re.sub(r"@\S+", " ", tweet)
    
    tweet = re.sub("[^a-zA-Z]", " ",tweet)
    
        
    text.append(tweet)
    
    return text
    

In [6]:
def slangword(tweet):
    convert = pd.read_csv('dataslang.csv', sep=',' ,header=None, names=['slang', 'fix'])
    slangword= []
    slang=[]
    
    for i in (convert['slang']):
        slangword.append(i)

   
   
    i=tweet.split()
    
    temp = ''

    for j in i:
        if (j in slangword):
            for c in range(len(convert)):
                if(convert['slang'][c] == j):
                    hasil = convert['fix'][c]
                    temp= temp + ' ' + hasil
                else:
                    temp = temp
                
        else:       
            temp= temp + ' '+ j
                
    slang.append(temp)
    return slang

In [7]:
def negation(kotak):
    negasi = ['tidak', 'bukan']
    wadah = []

    for i in kotak:
        
        i=i.split()
        n= 0
        k=0
        temp=''
        p = 0
        
        for j in i:
            p+=1
           
            if j in negasi and p != len(i):
                n= n+1
                k= n                
                hasil = j + i[n]
                temp= temp + ' '+ hasil            
            elif j == i[k] and n!= 0 :
                n= n+1
                temp= temp                
            else:
                n= n+1
                temp= temp +' '+ j
         
        wadah.append(temp)
    
    return wadah   

In [12]:
def stopword(wadah):
    
    factory = StopWordRemoverFactory()
    more_stopword = ['dengan', 'ia','bahwa','oleh','aku','kamu','bukan']
    data =factory.get_stop_words()+more_stopword
    stopword = factory.create_stop_word_remover()
    stemmer = StemmerFactory()
    stemmer = stemmer.create_stemmer()
    # Kalimat
    
    for i in wadah:
        stop=(word_tokenize(stemmer.stem(stopword.remove(i))))
    return stop

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [25]:
wadah = ['memuaskan']  
factory = StopWordRemoverFactory()
more_stopword = ['dengan', 'ia','bahwa','oleh','aku','kamu','bukan']
data =factory.get_stop_words()+more_stopword
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory()
stemmer = stemmer.create_stemmer()
# Kalimat
    
for i in wadah:
    stop=(word_tokenize(stemmer.stem(stopword.remove(i))))
stop

['muas']

In [9]:
def gabung (tweet):
    text=[]
    for j in tweet:
        spasi = ''
        for i in j:
            spasi = spasi + ' ' + i
        text.append(spasi)
    return text

In [10]:
def clear(data):
    wadah=[]
    for i in data:
        clean = cleaning(i)
        #print(clean)
        #slang = slangword(clean)
        #print(slang)
        #negasi = negation(slang)
        #print(negasi)
        stop = stopword(clean)
        #print(stop)
        wadah.append(stop)
    #print(wadah)
    return wadah
    

In [11]:
def preprocessing(wadah):
    data= clear(wadah)
    #print(data)
    hasil = gabung(data)
    #print(hasil)
   
    return hasil

In [12]:
X_train = preprocessing(X)

In [13]:
X_train

[' susah ngomong sama orang ga tau diri',
 ' macam mw balas tweet klau sy x pandai english haiszz sedih',
 ' knp ka aku jg gk denger siar trakhir kk lg sedih mksih ka udh jd tmn aku stiap kk lg siar',
 ' hujan lbat petirr aku takut ken off aja',
 ' tadi tenang rumah skrang malah jadi takut ga tenang',
 ' asa sedih bila di tempat beda',
 ' suka bnget pergi bilang bikin orang marah aja sih kmu',
 ' percuma beli bh mahal mahal kalo isi murah',
 ' yeah menang rentas desa gembira gila batak kejap',
 ' hjn abu nya smpe sekolah ku takut',
 ' ga ngomel ngomel',
 ' lah ko ketawa masalah',
 ' makan sehat hidup senang',
 ' pas gua udah mulai suka udah buka hati lu malah ngecewain takut',
 ' ku sedih banget bulan aku udah uts jadi gak ke jakarta huhuhu ken banget ktmu kalex',
 ' aten rindu sampai letak gmbar dua saya wechat gembira haru rindu tangisanterharugembirasemuaada',
 ' gue ga duli lo mau ngomong apa',
 ' pinter boong suka ngeles pinter cari muka otak kecil yah si budakkecikbalita marah',


# Multinomial Naive Bayes

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mutual_info_score
import math


In [20]:
def count_word_in_class(x,y):
    count = {}
    count_word = {}
    word_class = {}
    for i in range(0, len(x)):
        x_split = x[i].split()
        for word in x_split:
            w_c = word+str(",")+str(y[i])
            if (w_c in count):
                count[w_c] += 1
            else:
                count[w_c] = 1
                
            if (word in count_word):
                count_word[word] += 1
            else:
                count_word[word] = 1
                
            if (y[i] in word_class):
                word_class[y[i]] += 1
            else:
                word_class[y[i]] = 1
    return count, count_word, word_class

def count_class_in_doc(y):
    count = {}
    for i in range(0, len(y)):
        if (y[i] in count):
            count[y[i]] += 1
        else:
            count[y[i]] = 1
    return count

def hitung_prob(count,word,uniq):
    
    return ((count+1)/(word+uniq))

In [21]:
def multiNB(x,y,x_test):
    count_word_on_class, count_word, word_class = count_word_in_class(x,y)

    count_class = count_class_in_doc(y)

    kelaz = list(count_class.keys())
    y_pred = []
    for i in range(0,len(x_test)):
        prob_kelas = {}
        for j in count_class.keys():
            prob_c = 1
            x_split = x_test[i].split()
            for k in x_split:
                tag = k+str(',')+str(j)
                prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
            nama = str(i)+','+str(j)
            prob_kelas[nama] = prob_c
        y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    return y_pred

In [22]:
def get_subset(dataSet, indexes):
    """
    Selects subset of data based on indexes given
    from KFold validation method
    Returns:
        Subset of data set
    """
    return [dataSet[i] for i in indexes]

In [31]:
from sklearn.cross_validation import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn import metrics
kf = KFold(n=len(X_train), n_folds=10, shuffle=True)
sumakurasi = 0
for train_index, test_index in kf:
    features_train = get_subset(X_train, train_index)
    labels_train = get_subset(y, train_index)
    features_test = get_subset(X_train, test_index)
    labels_test = get_subset(y, test_index)
    
    akurasi = []
    
    y_pred_nb = multiNB(features_train, labels_train,  features_test)
    acc_nb = accuracy_score(labels_test,  y_pred_nb)*100
    sumakurasi += acc_nb
    report_nb = classification_report(labels_test, y_pred_nb)
    akurasi.append(acc_nb)
    print('++++')
    #print(y_pred_nb)
    print(acc_nb)
    
    print("Report 1: Naive Bayes")
    print(report_nb)
    
print('====')
print('rata akurasi: ' ,(sumakurasi/10))


++++
88.0
Report 1: Naive Bayes
             precision    recall  f1-score   support

      anger       0.90      0.78      0.84        23
       fear       0.95      0.95      0.95        20
        joy       0.87      0.93      0.90        29
    sadness       0.83      0.86      0.84        28

avg / total       0.88      0.88      0.88       100

++++
87.0
Report 1: Naive Bayes
             precision    recall  f1-score   support

      anger       0.82      0.92      0.87        25
       fear       0.93      0.90      0.91        29
        joy       0.86      0.86      0.86        22
    sadness       0.86      0.79      0.83        24

avg / total       0.87      0.87      0.87       100

++++
90.0
Report 1: Naive Bayes
             precision    recall  f1-score   support

      anger       0.89      0.93      0.91        27
       fear       0.96      0.92      0.94        24
        joy       1.00      0.86      0.92        28
    sadness       0.76      0.90      0.83       